In [2]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import shutil
from google.colab import files


# # Upload your Kaggle API key (kaggle.json)
files.upload()

kaggle_path = '/root/.config/kaggle'
os.makedirs(kaggle_path, exist_ok=True)
shutil.move('kaggle.json', os.path.join(kaggle_path,'kaggle.json'))

os.chmod(os.path.join(kaggle_path,'kaggle.json'), 600)
print("kaggle.json has been moved and configured.")



ModuleNotFoundError: No module named 'google'

In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Set up the Kaggle API client
api = KaggleApi()

api.authenticate()

dataset = "litvinenko630/insurance-claims"
download_dir = 'Insurance claims data.csv'

os.makedirs(download_dir,exist_ok=True)
api.dataset_download_files(dataset, path=download_dir, unzip=True)

print("Dataset Downloaded and extracted")

OSError: Could not find kaggle.json. Make sure it's located in C:\Users\kunja\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [6]:
import pandas as pd


data = pd.read_csv('Insurance claims data.csv')
data.head()

,policy_id,subscription_length,vehicle_age,customer_age,region_code,region_density,segment,model,fuel_type,max_torque,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status
0,POL045360,9.3,1.2,41,C8,8794,C2,M4,Diesel,250Nm@2750rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
1,POL016745,8.2,1.8,35,C2,27003,C1,M9,Diesel,200Nm@1750rpm,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,4,0
2,POL007194,9.5,0.2,44,C8,8794,C2,M4,Diesel,250Nm@2750rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
3,POL018146,5.2,0.4,44,C10,73430,A,M1,CNG,60Nm@3500rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
4,POL049011,10.1,1.0,56,C13,5410,B2,M5,Diesel,200Nm@3000rpm,...,No,Yes,Yes,Yes,No,No,Yes,Yes,5,0


In [7]:
#Preprocessing
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58592 entries, 0 to 58591
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   policy_id                         58592 non-null  object 
 1   subscription_length               58592 non-null  float64
 2   vehicle_age                       58592 non-null  float64
 3   customer_age                      58592 non-null  int64  
 4   region_code                       58592 non-null  object 
 5   region_density                    58592 non-null  int64  
 6   segment                           58592 non-null  object 
 7   model                             58592 non-null  object 
 8   fuel_type                         58592 non-null  object 
 9   max_torque                        58592 non-null  object 
 10  max_power                         58592 non-null  object 
 11  engine_type                       58592 non-null  object 
 12  airb

In [8]:
data_copy = data.copy()


#Dropping Policy ID column
columns_to_drop = ['policy_id']
data_copy.drop(columns=columns_to_drop, inplace=True)

data_copy.head()

,subscription_length,vehicle_age,customer_age,region_code,region_density,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status
0,9.3,1.2,41,C8,8794,C2,M4,Diesel,250Nm@2750rpm,113.45bhp@4000rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
1,8.2,1.8,35,C2,27003,C1,M9,Diesel,200Nm@1750rpm,97.89bhp@3600rpm,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,4,0
2,9.5,0.2,44,C8,8794,C2,M4,Diesel,250Nm@2750rpm,113.45bhp@4000rpm,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
3,5.2,0.4,44,C10,73430,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
4,10.1,1.0,56,C13,5410,B2,M5,Diesel,200Nm@3000rpm,88.77bhp@4000rpm,...,No,Yes,Yes,Yes,No,No,Yes,Yes,5,0


In [9]:
print(data_copy['claim_status'].unique())
print(data_copy['is_esc'].unique())

[0 1]
['Yes' 'No']


In [10]:
#changing the binary columns to Boolean

binary_columns = [col for col in data_copy.columns if data_copy[col].nunique() == 2 and data_copy[col].dtype == 'object']
print(binary_columns)


for col in binary_columns:
  data_copy[col] = data_copy[col].map({'Yes':True, 'No':False})


data_copy.info()

['is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors', 'is_parking_camera', 'rear_brakes_type', 'transmission_type', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58592 entries, 0 to 58591
Data columns (total 40 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   subscription_length               58592 non-null  float64
 1   vehicle_age                       58592 non-null  float64
 2   customer_age                      58592 non-null  int64  
 3   region_code                       58592 non-null  object 
 4   region_density                    58592 non-null  int64  
 5   segment                         

In [11]:
#Extracting numeric Values

data_copy['max_torque'] = data_copy['max_torque'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
data_copy['max_power'] = data_copy['max_power'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

data_copy.head()

,subscription_length,vehicle_age,customer_age,region_code,region_density,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status
0,9.3,1.2,41,C8,8794,C2,M4,Diesel,250.0,113.45,...,True,True,True,True,True,False,True,True,3,0
1,8.2,1.8,35,C2,27003,C1,M9,Diesel,200.0,97.89,...,False,True,True,True,True,True,True,True,4,0
2,9.5,0.2,44,C8,8794,C2,M4,Diesel,250.0,113.45,...,True,True,True,True,True,False,True,True,3,0
3,5.2,0.4,44,C10,73430,A,M1,CNG,60.0,40.36,...,False,False,False,True,False,False,False,True,0,0
4,10.1,1.0,56,C13,5410,B2,M5,Diesel,200.0,88.77,...,False,True,True,True,False,False,True,True,5,0


In [12]:
data['length_to_width_ratio'] = data['length'] / data['width']
data_copy.head()

,subscription_length,vehicle_age,customer_age,region_code,region_density,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status
0,9.3,1.2,41,C8,8794,C2,M4,Diesel,250.0,113.45,...,True,True,True,True,True,False,True,True,3,0
1,8.2,1.8,35,C2,27003,C1,M9,Diesel,200.0,97.89,...,False,True,True,True,True,True,True,True,4,0
2,9.5,0.2,44,C8,8794,C2,M4,Diesel,250.0,113.45,...,True,True,True,True,True,False,True,True,3,0
3,5.2,0.4,44,C10,73430,A,M1,CNG,60.0,40.36,...,False,False,False,True,False,False,False,True,0,0
4,10.1,1.0,56,C13,5410,B2,M5,Diesel,200.0,88.77,...,False,True,True,True,False,False,True,True,5,0


In [13]:
continous_col = ['subscription_length','turning_radius']
for col in continous_col:
  data_copy[col] = (data_copy[col] - data_copy[col].mean()) / data_copy[col].std()

data_copy.head(10)

,subscription_length,vehicle_age,customer_age,region_code,region_density,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,claim_status
0,0.769605,1.2,41,C8,8794,C2,M4,Diesel,250.0,113.45,...,True,True,True,True,True,False,True,True,3,0
1,0.504084,1.8,35,C2,27003,C1,M9,Diesel,200.0,97.89,...,False,True,True,True,True,True,True,True,4,0
2,0.817882,0.2,44,C8,8794,C2,M4,Diesel,250.0,113.45,...,True,True,True,True,True,False,True,True,3,0
3,-0.220066,0.4,44,C10,73430,A,M1,CNG,60.0,40.36,...,False,False,False,True,False,False,False,True,0,0
4,0.962712,1.0,56,C13,5410,B2,M5,Diesel,200.0,88.77,...,False,True,True,True,False,False,True,True,5,0
5,-0.726971,2.0,36,C7,6112,B2,M7,Petrol,113.0,88.50,...,True,True,True,True,True,True,True,True,0,0
6,-0.389034,2.4,38,C2,27003,C2,M4,Diesel,250.0,113.45,...,True,True,True,True,True,False,True,True,3,0
7,1.107542,2.0,56,C2,27003,B2,M6,Petrol,113.0,88.50,...,True,True,True,True,True,True,True,True,2,0
8,1.107542,0.6,55,C5,34738,B1,M8,CNG,82.1,55.92,...,False,True,True,True,False,False,True,True,2,0
9,-1.402844,2.4,45,C3,4076,B2,M6,Petrol,113.0,88.50,...,True,True,True,True,True,True,True,True,2,0


In [14]:
categorical_columns = ['region_code','fuel_type','segment','model','engine_type','transmission_type','steering_type']
data_copy = pd.get_dummies(data_copy, columns=categorical_columns, drop_first=True)


data_copy.head()

,subscription_length,vehicle_age,customer_age,region_density,max_torque,max_power,airbags,is_esc,is_adjustable_steering,is_tpms,...,engine_type_1.5 L U2 CRDi,engine_type_1.5 Turbocharged Revotorq,engine_type_1.5 Turbocharged Revotron,engine_type_F8D Petrol Engine,engine_type_G12B,engine_type_K Series Dual jet,engine_type_K10C,engine_type_i-DTEC,steering_type_Manual,steering_type_Power
0,0.769605,1.2,41,8794,250.0,113.45,6,True,True,True,...,True,False,False,False,False,False,False,False,False,True
1,0.504084,1.8,35,27003,200.0,97.89,2,False,True,False,...,False,False,False,False,False,False,False,True,False,False
2,0.817882,0.2,44,8794,250.0,113.45,6,True,True,True,...,True,False,False,False,False,False,False,False,False,True
3,-0.220066,0.4,44,73430,60.0,40.36,2,False,False,False,...,False,False,False,True,False,False,False,False,False,True
4,0.962712,1.0,56,5410,200.0,88.77,2,False,True,False,...,False,True,False,False,False,False,False,False,False,False


In [15]:
data_copy.info()

data_copy.columns = data_copy.columns.str.replace(r'[.\s]', '_', regex=True)


# Mapping dataset headers to SQL table headers
column_mapping = {
    'engine_type_1_2_L_K_Series_Engine': 'engine_type_1_2_l_k_series_engine',
    'engine_type_1_2_L_K12N_Dualjet': 'engine_type_1_2_l_k12n_dualjet',
    'engine_type_1_5_L_U2_CRDi': 'engine_type_1_5_l_u2_crdi',
    'engine_type_1_5_Turbocharged_Revotorq': 'engine_type_1_5_turbocharged_revotorq',
    'engine_type_1_5_Turbocharged_Revotron': 'engine_type_1_5_turbocharged_revotron',
    'engine_type_F8D_Petrol_Engine': 'engine_type_f8d_petrol_engine',
    'engine_type_G12B': 'engine_type_g12b',
    'engine_type_K_Series_Dual_jet': 'engine_type_k_series_dual_jet',
    'engine_type_K10C': 'engine_type_k10c',
    'engine_type_i-DTEC': 'engine_type_i_dtec'
}

# Rename dataset columns
data_copy = data_copy.rename(columns=column_mapping)

# Verify the renamed columns
print(data_copy.columns)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58592 entries, 0 to 58591
Data columns (total 83 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   subscription_length                    58592 non-null  float64
 1   vehicle_age                            58592 non-null  float64
 2   customer_age                           58592 non-null  int64  
 3   region_density                         58592 non-null  int64  
 4   max_torque                             58592 non-null  float64
 5   max_power                              58592 non-null  float64
 6   airbags                                58592 non-null  int64  
 7   is_esc                                 58592 non-null  bool   
 8   is_adjustable_steering                 58592 non-null  bool   
 9   is_tpms                                58592 non-null  bool   
 10  is_parking_sensors                     58592 non-null  bool   
 11  is

In [17]:
cleansed_file_path = 'Cleansed_insurance_claims_data (1).csv'
data_copy.to_csv(cleansed_file_path, index=False)

**SQL**

In [18]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
print(data_copy['is_esc'].unique())


print(data_copy.columns)

[ True False]
Index(['subscription_length', 'vehicle_age', 'customer_age', 'region_density',
       'max_torque', 'max_power', 'airbags', 'is_esc',
       'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
       'is_parking_camera', 'rear_brakes_type', 'displacement', 'cylinder',
       'turning_radius', 'length', 'width', 'gross_weight',
       'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer',
       'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks',
       'is_central_locking', 'is_power_steering',
       'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
       'is_ecw', 'is_speed_alert', 'ncap_rating', 'claim_status',
       'region_code_C10', 'region_code_C11', 'region_code_C12',
       'region_code_C13', 'region_code_C14', 'region_code_C15',
       'region_code_C16', 'region_code_C17', 'region_code_C18',
       'region_code_C19', 'region_code_C2', 'region_code_C20',
       'region_code_C21', 'region_code_C22', 're

In [33]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname = 'InsuranceWarehouse',
    user = 'postgres',
    password = 'your password',
    host = 'localhost',
    port ='5432'
)

cursor = conn.cursor()


customers = data_copy[['customer_age','region_density']].drop_duplicates()

for _, row in customers.iterrows():
    cursor.execute(
        "INSERT INTO Customers (customer_age,region_density) VALUES (%s, %s)",
        (int(row['customer_age']),int(row['region_density']))
    )

vehicles = data_copy[['vehicle_age','fuel_type_Diesel','fuel_type_Petrol','airbags','displacement','max_torque','max_power']].drop_duplicates()

for _, row in vehicles.iterrows():
    cursor.execute(
        "INSERT INTO Vehicles (vehicle_age, fuel_type, airbags, displacement, max_torque, max_power) VALUES ( %s, %s, %s, %s, %s, %s)",
        (row['vehicle_age'], 'Diesel' if row['fuel_type_Diesel'] else 'Petrol', row['airbags'],row['displacement'],row['max_torque'],row['max_power'])
    )

features = data_copy[['is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors', 'is_parking_camera', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']].drop_duplicates()
for _, row in features.iterrows():
    cursor.execute(
        """
        INSERT INTO Features (is_esc, is_adjustable_steering, is_tpms, is_parking_sensors, is_parking_camera, is_front_fog_lights, is_rear_window_wiper, is_rear_window_washer, is_rear_window_defogger, is_brake_assist, is_power_door_locks, is_central_locking, is_power_steering, is_driver_seat_height_adjustable, is_day_night_rear_view_mirror, is_ecw, is_speed_alert)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        tuple(row)
    )

regions = data_copy[['region_code_C10', 'region_code_C11', 'region_code_C12', 'region_code_C13', 'region_code_C14', 'region_code_C15', 'region_code_C16', 'region_code_C17', 'region_code_C18', 'region_code_C19', 'region_code_C2', 'region_code_C20', 'region_code_C21', 'region_code_C22', 'region_code_C3', 'region_code_C4', 'region_code_C5', 'region_code_C6', 'region_code_C7', 'region_code_C8', 'region_code_C9']].drop_duplicates()
for _, row in regions.iterrows():
    cursor.execute(
        """
        INSERT INTO Regions (region_code_c10, region_code_c11, region_code_c12, region_code_c13, region_code_c14, region_code_c15, region_code_c16, region_code_c17, region_code_c18, region_code_c19, region_code_c2, region_code_c20, region_code_c21, region_code_c22, region_code_c3, region_code_c4, region_code_c5, region_code_c6, region_code_c7, region_code_c8, region_code_c9)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        tuple(row)
    )

models = data_copy[['model_M2','model_M3','model_M4','model_M5','model_M6','model_M7','model_M8','model_M9','model_M10','model_M11']]

for _,row in models.iterrows():
    cursor.execute(
        """
            INSERT INTO Models (model_m2, model_m3, model_m4, model_m5, model_m6, model_m7, model_m8, model_m9, model_m10, model_m11) VALUES
            (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """,
        tuple(row)
    )

engines = data_copy[[
    'engine_type_1_2_l_k_series_engine', 
    'engine_type_1_2_l_k12n_dualjet', 
    'engine_type_1_5_l_u2_crdi', 
    'engine_type_1_5_turbocharged_revotorq',
    'engine_type_1_5_turbocharged_revotron',
    'engine_type_f8d_petrol_engine',
    'engine_type_g12b', 
    'engine_type_k_series_dual_jet',
    'engine_type_k10c',
    'engine_type_i_dtec'
]].drop_duplicates()

# Insert data into the Engines table
for _, row in engines.iterrows():
    cursor.execute(
        """
        INSERT INTO Engines (
            engine_type_1_2_l_k_series_engine,
            engine_type_1_2_l_k12n_dualjet,
            engine_type_1_5_l_u2_crdi,
            engine_type_1_5_turbocharged_revotorq,
            engine_type_1_5_turbocharged_revotron,
            engine_type_f8d_petrol_engine,
            engine_type_g12b,
            engine_type_k_series_dual_jet,
            engine_type_k10c,
            engine_type_i_dtec
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        tuple(row)
    )

def get_dimension_id(field, table, columns, values):
    query = f"SELECT {field}_id FROM {table} WHERE " + " AND ".join([f"{col}= %s" for col in columns]) 
    cursor.execute(query,values)
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return None
    

for _,row in data_copy.iterrows():
    customer_id = get_dimension_id("customer","Customers",["customer_age","region_density"],[row['customer_age'],row['region_density']])
    vehicle_id = get_dimension_id('vehicle',"Vehicles", ["vehicle_age", "fuel_type", "airbags", "displacement"], [row["vehicle_age"], 'Diesel' if row['fuel_type_Diesel'] else 'Petrol', row["airbags"], row["displacement"]])
    feature_id = get_dimension_id('feature',"Features",[
        "is_esc", "is_adjustable_steering", "is_tpms", 
        "is_parking_sensors", "is_parking_camera", "is_front_fog_lights",
        "is_rear_window_wiper", "is_rear_window_washer", "is_rear_window_defogger", 
        "is_brake_assist", "is_power_door_locks", "is_central_locking", 
        "is_power_steering", "is_driver_seat_height_adjustable", 
        "is_day_night_rear_view_mirror", "is_ecw", "is_speed_alert"
    ],
    [
        row["is_esc"], row["is_adjustable_steering"], row["is_tpms"], 
        row["is_parking_sensors"], row["is_parking_camera"], row["is_front_fog_lights"],
        row["is_rear_window_wiper"], row["is_rear_window_washer"], row["is_rear_window_defogger"], 
        row["is_brake_assist"], row["is_power_door_locks"], row["is_central_locking"], 
        row["is_power_steering"], row["is_driver_seat_height_adjustable"], 
        row["is_day_night_rear_view_mirror"], row["is_ecw"], row["is_speed_alert"]
    ]
    )
    region_id = get_dimension_id('region',"Regions",[
        "region_code_C10", "region_code_C11", "region_code_C12", 
        "region_code_C13", "region_code_C14", "region_code_C15", 
        "region_code_C16", "region_code_C17", "region_code_C18", 
        "region_code_C19", "region_code_C2", "region_code_C20", 
        "region_code_C21", "region_code_C22", "region_code_C3", 
        "region_code_C4", "region_code_C5", "region_code_C6", 
        "region_code_C7", "region_code_C8", "region_code_C9"
    ],
    [
        row["region_code_C10"], row["region_code_C11"], row["region_code_C12"], 
        row["region_code_C13"], row["region_code_C14"], row["region_code_C15"], 
        row["region_code_C16"], row["region_code_C17"], row["region_code_C18"], 
        row["region_code_C19"], row["region_code_C2"], row["region_code_C20"], 
        row["region_code_C21"], row["region_code_C22"], row["region_code_C3"], 
        row["region_code_C4"], row["region_code_C5"], row["region_code_C6"], 
        row["region_code_C7"], row["region_code_C8"], row["region_code_C9"]
    ]
    )
    engine_id = get_dimension_id('engine',"Engines", [
        "engine_type_1_2_l_k_series_engine", 
        "engine_type_1_2_l_k12n_dualjet", 
        "engine_type_1_5_l_u2_crdi", 
        "engine_type_1_5_turbocharged_revotorq",
        "engine_type_1_5_turbocharged_revotron",
        "engine_type_f8d_petrol_engine",
        "engine_type_g12b", 
        "engine_type_k_series_dual_jet",
        "engine_type_k10c",
        "engine_type_i_dtec"
    ],
    [
        row["engine_type_1_2_l_k_series_engine"], 
        row["engine_type_1_2_l_k12n_dualjet"], 
        row["engine_type_1_5_l_u2_crdi"], 
        row["engine_type_1_5_turbocharged_revotorq"],
        row["engine_type_1_5_turbocharged_revotron"],
        row["engine_type_f8d_petrol_engine"],
        row["engine_type_g12b"], 
        row["engine_type_k_series_dual_jet"],
        row["engine_type_k10c"],
        row["engine_type_i_dtec"]
    ]
    )
    model_id = get_dimension_id(
    'model',
    "Models",
    [
        "model_M10", "model_M11", "model_M2", 
        "model_M3", "model_M4", "model_M5", 
        "model_M6", "model_M7", "model_M8", "model_M9"
    ],
    [
        row["model_M10"], row["model_M11"], row["model_M2"], 
        row["model_M3"], row["model_M4"], row["model_M5"], 
        row["model_M6"], row["model_M7"], row["model_M8"], row["model_M9"]
    ]
    )

    if None in [customer_id, vehicle_id, feature_id, region_id, engine_id, model_id]:
        continue

    cursor.execute(
        """
        INSERT INTO Claims (subscription_length, claim_status, customer_id, vehicle_id, feature_id, region_id, engine_id, model_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (row["subscription_length"], row["claim_status"], customer_id, vehicle_id, feature_id, region_id, engine_id, model_id)
    )


# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("Engines table has been successfully populated.")

Engines table has been successfully populated.
